# Notebook: Prompt Generation and Configuration

This notebook validates prompt creation, context provider integrations, extension mechanisms, and logging behavior. It follows the structure of the linting build-up notebook to ensure consistent testing.

In [ ]:
import sys
from importlib import import_module
import logging
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

from app.factories.prompt_manager import PromptGeneratorFactory
from app.factories.context_provider import ContextProviderFactory
from app.factories.logging_provider import LoggingProvider
from app.factories.tool_provider import ToolProviderFactory

import_module('app.extensions.prompt_generators')
import_module('app.extensions.context_providers')
import_module('app.extensions.tool_providers')

logging.basicConfig(level=logging.DEBUG)

### Basic System Prompt

A minimal system prompt is created using the `BasicPromptGenerator`. The prompt combines system and agent templates with optional context.

In [ ]:
provider = ContextProviderFactory.create('dummy')
generator = PromptGeneratorFactory.create('basic', context_provider=provider)

system_config = {'template': 'System instructions'}
agent_config = {'template': 'Agent instructions'}

try:
    prompt = generator.generate_prompt(agent_config, system_config)
    print(prompt)
except Exception as e:
    print('Prompt generation failed:', e)

### Multiple Agent-Level Prompts

Prompts for generation, evaluation, and mediation roles are produced to ensure the generator handles varied agent instructions.

In [ ]:
agent_configs = {
    'generation': {'template': 'Generation prompt'},
    'evaluation': {'template': 'Evaluation prompt'},
    'mediation': {'template': 'Mediation prompt'},
}

for name, cfg in agent_configs.items():
    try:
        p = generator.generate_prompt(cfg, system_config)
        print(f'{name}: {p}')
    except Exception as e:
        print(f'{name} failed:', e)

### Context Providers

Different context providers supply additional code data. This cell registers available providers and renders prompts with each.

In [ ]:
dummy = ContextProviderFactory.create('dummy')
try:
    symbol = ContextProviderFactory.create('symbol_graph', target=Path('sample_module.py'))
except Exception as e:
    print('SymbolGraphProvider creation failed:', e)
    symbol = None

providers = [p for p in (dummy, symbol) if p is not None]

for p in providers:
    try:
        _ = p.get_context()
        result = PromptGeneratorFactory.create('basic', context_provider=p).generate_prompt(agent_config, system_config)
        print(f'{p.__class__.__name__}: {result}')
    except Exception as e:
        print(p.__class__.__name__, 'error:', e)

### Tool Providers

Tool providers execute external utilities used in the prompting workflow. Errors for invalid providers are explicitly shown.

In [ ]:
try:
    black = ToolProviderFactory.create('black')
    tmp = Path('tmp.py')
    tmp.write_text('x=1')
    result = black.run(str(tmp))
    print('Black return:', result.returncode)
except Exception as e:
    print('Black failed:', e)
finally:
    tmp.unlink(missing_ok=True)

try:
    ToolProviderFactory.create('nonexistent')
except Exception as e:
    print('Expected tool error:', e)

### Logging Verification

Logging operations should succeed under normal conditions and raise errors on misuse.

In [ ]:
logger = LoggingProvider(':memory:')
from app.enums.logging_enums import LogType
from app.utilities.metadata.logging.log_schemas import StateLog

try:
    logger.log_state(StateLog(experiment_id='exp', round=0, state='test'))
    print('State logged')
except Exception as e:
    print('Logging failed:', e)

try:
    logger.write(LogType.STATE, object())
except Exception as e:
    print('Expected logging error:', e)